In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from geopy.distance import great_circle
import seaborn as sns
import datetime

In [ ]:
train=pd.read_csv('/content/gdrive/My Drive/Datasets/NYC Taxi Kaggle/train.csv')
test=pd.read_csv('/content/gdrive/My Drive/Datasets/NYC Taxi Kaggle/test.csv')
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [ ]:
train['distance']=train.apply(lambda row: great_circle((row.pickup_latitude,row.pickup_longitude), (row.dropoff_latitude, row.dropoff_longitude)).km, axis=1)


In [ ]:
test['distance']=test.apply(lambda row: great_circle((row.pickup_latitude,row.pickup_longitude), (row.dropoff_latitude, row.dropoff_longitude)).km, axis=1)

In [ ]:
train['pickup_datetime']=pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime']=pd.to_datetime(train['dropoff_datetime'])

test['pickup_datetime']=pd.to_datetime(test['pickup_datetime'])


In [ ]:
train['pickup_day']=train['pickup_datetime'].dt.day_name()
train['dropoff_day']=train['dropoff_datetime'].dt.day_name()


test['pickup_day']=test['pickup_datetime'].dt.day_name()

In [ ]:
def timezone(x):
    if x>=datetime.time(4, 0, 1) and x <=datetime.time(10, 0, 0):
        return 'morning'
    elif x>=datetime.time(10, 0, 1) and x <=datetime.time(16, 0, 0):
        return 'midday'
    elif x>=datetime.time(16, 0, 1) and x <=datetime.time(22, 0, 0):
        return 'evening'
    elif x>=datetime.time(22, 0, 1) or x <=datetime.time(4, 0, 0):
        return 'late night'
    
train['pickup_timezone']=train['pickup_datetime'].apply(lambda x :timezone(datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").time()) )
train['dropoff_timezone']=train['dropoff_datetime'].apply(lambda x :timezone(datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").time()) )

test['pickup_timezone']=test['pickup_datetime'].apply(lambda x :timezone(datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").time()) )

In [ ]:
train=train[train.trip_duration!=train.trip_duration.max()]
train=train[train.trip_duration!=train.trip_duration.max()]
train=train[train.trip_duration!=train.trip_duration.max()]
train=train[train.trip_duration!=train.trip_duration.max()]
print( train['trip_duration'].nlargest(10))

1234291    86392
295382     86391
73816      86390
59891      86387
1360439    86385
753765     86379
91717      86378
1221666    86378
66346      86377
1138915    86377
Name: trip_duration, dtype: int64


In [ ]:
train.loc[train['store_and_fwd_flag']=='N', 'store_and_fwd_flag']=0
train.loc[train['store_and_fwd_flag']=='Y', 'store_and_fwd_flag']=1
test.loc[test['store_and_fwd_flag']=='N', 'store_and_fwd_flag']=0
test.loc[test['store_and_fwd_flag']=='Y', 'store_and_fwd_flag']=1



train.loc[train['pickup_day']=='Monday', 'pickup_day']=0
train.loc[train['pickup_day']=='Tuesday', 'pickup_day']=1
train.loc[train['pickup_day']=='Wednesday', 'pickup_day']=2
train.loc[train['pickup_day']=='Thursday', 'pickup_day']=3
train.loc[train['pickup_day']=='Friday', 'pickup_day']=4
train.loc[train['pickup_day']=='Saturday', 'pickup_day']=5
train.loc[train['pickup_day']=='Sunday', 'pickup_day']=6

test.loc[test['pickup_day']=='Monday', 'pickup_day']=0
test.loc[test['pickup_day']=='Tuesday', 'pickup_day']=1
test.loc[test['pickup_day']=='Wednesday', 'pickup_day']=2
test.loc[test['pickup_day']=='Thursday', 'pickup_day']=3
test.loc[test['pickup_day']=='Friday', 'pickup_day']=4
test.loc[test['pickup_day']=='Saturday', 'pickup_day']=5
test.loc[test['pickup_day']=='Sunday', 'pickup_day']=6

train.loc[train['dropoff_day']=='Monday', 'dropoff_day']=0
train.loc[train['dropoff_day']=='Tuesday', 'dropoff_day']=1
train.loc[train['dropoff_day']=='Wednesday', 'dropoff_day']=2
train.loc[train['dropoff_day']=='Thursday', 'dropoff_day']=3
train.loc[train['dropoff_day']=='Friday', 'dropoff_day']=4
train.loc[train['dropoff_day']=='Saturday', 'dropoff_day']=5
train.loc[train['dropoff_day']=='Sunday', 'dropoff_day']=6

train.loc[train['pickup_timezone']=='morning', 'pickup_timezone']=0
train.loc[train['pickup_timezone']=='midday', 'pickup_timezone']=1
train.loc[train['pickup_timezone']=='evening', 'pickup_timezone']=2
train.loc[train['pickup_timezone']=='late night', 'pickup_timezone']=3

test.loc[test['pickup_timezone']=='morning', 'pickup_timezone']=0
test.loc[test['pickup_timezone']=='midday', 'pickup_timezone']=1
test.loc[test['pickup_timezone']=='evening', 'pickup_timezone']=2
test.loc[test['pickup_timezone']=='late night', 'pickup_timezone']=3

train.loc[train['dropoff_timezone']=='morning', 'dropoff_timezone']=0
train.loc[train['dropoff_timezone']=='midday', 'dropoff_timezone']=1
train.loc[train['dropoff_timezone']=='evening', 'dropoff_timezone']=2
train.loc[train['dropoff_timezone']=='late night', 'dropoff_timezone']=3

In [ ]:
train.drop(['id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude','pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)


In [ ]:
train.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance,pickup_day,dropoff_day,pickup_timezone,dropoff_timezone
0,2,1,0,455,1.498523,0,0,2,2
1,1,1,0,663,1.805510,6,6,3,3
2,2,1,0,2124,6.385108,1,1,1,1
3,2,1,0,429,1.485501,2,2,2,2
4,2,1,0,435,1.188590,5,5,1,1


In [ ]:
X_train=train.drop(['trip_duration', 'dropoff_timezone', 'dropoff_day'], axis=1).copy()
Y_train= train['trip_duration']


In [ ]:
X_test=test.drop(['id', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1).copy()

In [ ]:
X_test.head()


,vendor_id,passenger_count,store_and_fwd_flag,distance,pickup_day,pickup_timezone
0,1,1,0,2.746430,3,3
1,1,1,0,2.759243,3,3
2,1,1,0,1.306157,3,3
3,2,1,0,5.269095,3,3
4,1,1,0,0.960843,3,3


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(32, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))


# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 32)                224       
_________________________________________________________________
dense_12 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_13 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_14 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 464,737
Trainable params: 464,737
Non-trainable params: 0
________________________________________________

In [ ]:
NN_model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split = 0.2)

Train on 1166912 samples, validate on 291728 samples
Epoch 1/10
1166912/1166912 [==============================] - 179s 153us/step - loss: 0.2416 - acc: 0.0017 - val_loss: 0.2371 - val_acc: 0.0018
Epoch 2/10
1166912/1166912 [==============================] - 181s 155us/step - loss: 0.2361 - acc: 0.0018 - val_loss: 0.2350 - val_acc: 0.0018
Epoch 3/10
1166912/1166912 [==============================] - 181s 155us/step - loss: 0.2353 - acc: 0.0018 - val_loss: 0.2357 - val_acc: 0.0018
Epoch 4/10
1166912/1166912 [==============================] - 177s 151us/step - loss: 0.2345 - acc: 0.0018 - val_loss: 0.2345 - val_acc: 0.0020
Epoch 5/10
1166912/1166912 [==============================] - 174s 149us/step - loss: 0.2340 - acc: 0.0018 - val_loss: 0.2347 - val_acc: 0.0018
Epoch 6/10
1166912/1166912 [==============================] - 172s 148us/step - loss: 0.2335 - acc: 0.0018 - val_loss: 0.2336 - val_acc: 0.0018
Epoch 7/10
1166912/1166912 [==============================] - 174s 149us/step - los

In [ ]:
X_test.drop(['prediction'], axis=1)

,vendor_id,passenger_count,store_and_fwd_flag,distance,pickup_day,pickup_timezone
0,1,1,0,2.746430,3,3
1,1,1,0,2.759243,3,3
2,1,1,0,1.306157,3,3
3,2,1,0,5.269095,3,3
4,1,1,0,0.960843,3,3
...,...,...,...,...,...,...
625129,1,1,0,0.967439,4,3
625130,1,1,0,6.196352,4,3
625131,1,2,0,8.153731,4,3
625132,1,1,0,20.380777,4,3


In [ ]:
X_test.drop(['prediction'], axis=1)

,vendor_id,passenger_count,store_and_fwd_flag,distance,pickup_day,pickup_timezone
0,1,1,0,2.746430,3,3
1,1,1,0,2.759243,3,3
2,1,1,0,1.306157,3,3
3,2,1,0,5.269095,3,3
4,1,1,0,0.960843,3,3
...,...,...,...,...,...,...
625129,1,1,0,0.967439,4,3
625130,1,1,0,6.196352,4,3
625131,1,2,0,8.153731,4,3
625132,1,1,0,20.380777,4,3


In [ ]:
print(X_train.columns)
print(X_test.columns)

Index(['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance',
       'pickup_day', 'pickup_timezone'],
      dtype='object')
Index(['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance',
       'pickup_day', 'pickup_timezone', 'prediction'],
      dtype='object')


In [ ]:
predictions = NN_model.predict(X_test)

In [ ]:
test['prediction']=predictions


In [ ]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,distance,pickup_day,pickup_timezone,prediction
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,0,2.746430,3,3,599.355408
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,0,2.759243,3,3,601.393311
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,0,1.306157,3,3,356.881287
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,0,5.269095,3,3,981.785034
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,0,0.960843,3,3,281.089905


In [ ]:
test.drop(['vendor_id', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'distance', 'pickup_day', 'pickup_timezone'], axis=1).copy()


,id,prediction
0,id3004672,670.474304
1,id3505355,672.670715
2,id1217141,380.866882
3,id2150126,1111.261230
4,id1598245,296.670227
...,...,...
625129,id3008929,314.968384
625130,id3700764,1233.037720
625131,id2568735,1526.666260
625132,id1384355,2173.610352


In [ ]:
submission=pd.DataFrame({"id": test['id'], "trip_duration": test['prediction']})
submission.to_csv('submission_NN_100.csv')